In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.io_utils import *
from src.metrics import *

df = read_data("data/ecommerce_sales.csv")

df_clean = clean_data(df)

print("---" * 50)
print()
print("Here is a sample of our data:")
print()
print(df_clean.sample(3))
print()
print("---" * 50)

------------------------------------------------------------------------------------------------------------------------------------------------------

Here is a sample of our data:

      order_id       date       city category   price  units  revenue
743     100743 2024-03-07   Göteborg     Home  584.80      4  2339.20
1676    101676 2024-01-21   Göteborg     Home  412.99      2   825.98
1626    101626 2024-02-24  Stockholm   Sports  413.93      4  1655.72

------------------------------------------------------------------------------------------------------------------------------------------------------


## Total revenue and units sold

In [5]:
print("---" * 50)
print()
print(f"In 2024 we sold a total of {total_units(df_clean)} units,\n"
      f"for a total revenue of {total_revenue(df_clean)} sek.")
print()
print("---" * 50)

------------------------------------------------------------------------------------------------------------------------------------------------------

In 2024 we sold a total of 7463 units,
for a total revenue of 3502767.23 sek.

------------------------------------------------------------------------------------------------------------------------------------------------------


## Average Order

In [9]:
print("---" * 50)
print()
print(f"Th average order is {average_order(df_clean)} sek.")
print()
print("---" * 50)

------------------------------------------------------------------------------------------------------------------------------------------------------

Th average order is 1401.11 sek.

------------------------------------------------------------------------------------------------------------------------------------------------------
